# Evaluating Neural Net Predictions

Remember to change the `model_type` and `ver` variables prior to running!

In [ ]:
import sys
sys.path.append('C:/Users/chetai/Documents/Projects/moonGen/')

import os
import pdb
import json
import pickle

import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

from scripts.evaluation.eval_utils import *
from scripts.evaluation.evaluation_tools import *
from scripts.pytorch.utils.train_test_functions import *

## Setup Experiment Paths

In [ ]:
# Specify experiment settings
model_type = 'GCN'
ver = 'v0'

data_dir = 'C:/Users/chetai/Desktop/moonboard_data/'
result_dir = 'C:/Users/chetai/Desktop/moonboard_results/'

subpath_suffix = '%s/%s/' % (model_type, ver)
data_path = data_dir + subpath_suffix
result_path = result_dir + subpath_suffix

# Load experiment parameters
with open(data_path + 'params.json', 'r') as f:
    param_dict = json.load(f)

In [ ]:
# Define paths to model outputs
model_path = result_path + 'model.pickle'
features_path = result_path + 'features.pickle'
adj_path = result_path + 'adj.pickle'
labels_path = result_path + 'labels.pickle'
idx_train_path = result_path + 'idx_train.pickle'
idx_val_path = result_path + 'idx_dev.pickle'
idx_test_path = result_path + 'idx_test.pickle'

# Load model outputs
model = load_pickle(model_path)
features = load_pickle(features_path)
adj = load_pickle(adj_path)
labels = load_pickle(labels_path)
idx_train = load_pickle(idx_train_path)
idx_val = load_pickle(idx_val_path)
idx_test = load_pickle(idx_test_path)

## Inference

In [ ]:
# Get accuracy on test set
test_dict = dict()
test_dict['features'] = features
test_dict['adj'] = adj
test_dict['labels'] = labels
test_dict['idx_test'] = idx_test

test(model, test_dict)

In [ ]:
# Forward pass on network
output = model(features, adj)

In [ ]:
# Test predictions
Y_pred_test = np.exp(output[idx_test].detach().numpy())[:, 1:]  # Drop 0 class (holds)
Y_label_test = onehot_labels(labels.numpy()[idx_test] - 1, Y_pred_test.shape[1])  # Shift labels by 1

In [ ]:
# Train predictions
Y_pred_train = np.exp(output[idx_train].detach().numpy())[:, 1:]  
Y_label_train = onehot_labels(labels.numpy()[idx_train] - 1, Y_pred_train.shape[1])

## Generate Evaluation Plots

In [ ]:
trn_out_root = result_path + 'train/'
tst_out_root = result_path + 'test/'

make_directory(trn_out_root)
make_directory(tst_out_root)
    
trn_settings = {
    'description': 'Train Debug',
    'corr_fig_save': trn_out_root + 'fig_correlation.png',
    'farp_fig_save': trn_out_root + 'fig_farpa.png',
    'farp_stats_save': trn_out_root + 'stats_farpa.pickle',
    'confusion_fig_save': trn_out_root + 'fig_confusion.png',
    'global_stats_save': trn_out_root + 'stats_global.pickle'
}

tst_settings = {
    'description': 'Test Debug',
    'corr_fig_save': tst_out_root + 'fig_correlation.png',
    'farp_fig_save': tst_out_root + 'fig_farpa.png',
    'farp_stats_save': tst_out_root + 'stats_farpa.pickle',
    'confusion_fig_save': tst_out_root + 'fig_confusion.png',
    'global_stats_save': tst_out_root + 'stats_global.pickle'
}

In [ ]:
evaluate_predictions(Y_label_train, Y_pred_train, trn_settings)

In [ ]:
evaluate_predictions(Y_label_test, Y_pred_test, tst_settings)